In [58]:
import pandas as pd
import ast
import json


In [4]:
paid = pd.read_csv('Данные/paid_all_period.csv')
affected = pd.read_csv('Данные/sectors_cluster_affected_total_all_period.csv')
targeted = pd.read_csv('Данные/sectors_cluster_targeted_total_all_period.csv')

In [6]:
paid.head()

,Unnamed: 0,Year,Donor Name,CBPF Name,Paid Amount,Pledged Amount,Total Contributions,Local Curency,Exchange Rate,Paid Amount (Local Currency),Pledged Amount (Local Currency),Total Contributions (Local Currency)
0,0,2006,Netherlands,Ethiopia,5012000.0,0.0,5012000.0,USD,1.00,5012000.00,0,5012000.00
1,1,2006,Norway,Ethiopia,1553277.0,0.0,1553277.0,USD,1.00,1553277.00,0,1553277.00
2,2,2006,Switzerland,Ethiopia,247934.0,0.0,247934.0,CHF,1.21,300000.14,0,300000.14
3,3,2006,United Kingdom,Ethiopia,8861284.0,0.0,8861284.0,GBP,0.56,5000000.20,0,5000000.20
4,0,2007,Netherlands,Ethiopia,3395940.0,0.0,3395940.0,USD,1.00,3395940.00,0,3395940.00


In [17]:
paid_money_by_country = pd.DataFrame(paid['CBPF Name'].value_counts())
paid_money_by_country['total_contributions'] = paid.groupby(by=['CBPF Name']).sum()['Total Contributions']
paid_money_by_country.sort_values(by='total_contributions')


,CBPF Name,total_contributions
Niger (RhPF),6,8.764908e+06
Burkina Faso (RhPF),6,1.228212e+07
Colombia,24,1.270318e+07
Venezuela,11,1.276383e+07
Jordan,35,6.183625e+07
Myanmar,65,9.210216e+07
Haiti,47,9.386667e+07
Pakistan,33,1.033629e+08
Lebanon,60,1.105090e+08
Nigeria,61,1.512291e+08


In [18]:
paid_money_by_donor = pd.DataFrame(paid['Donor Name'].value_counts())
paid_money_by_donor['total_contributions'] = paid.groupby(by=['Donor Name']).sum()['Total Contributions']
paid_money_by_donor.sort_values(by='total_contributions')

,Donor Name,total_contributions
Kenya,1,1.987330e+03
Madagascar,1,5.000000e+03
Sri Lanka,4,5.000000e+03
Chile,1,5.000000e+03
Montenegro,1,1.234568e+04
...,...,...
Norway,115,4.384337e+08
Netherlands,97,7.962476e+08
Sweden,177,9.000405e+08
Germany,111,1.073204e+09


In [132]:
def count_degree_paid(data, 
                      affected,
                      targeted,
                      year_begin=2020, 
                      year_finish=2021, 
                      count_type='unique_country'):
    

    data_period = data[(data.Year >= year_begin) & (data.Year <= year_finish)]
    affected = affected[(affected.Year >= year_begin) & (affected.Year <= year_finish)]
    targeted = targeted[(targeted.Year >= year_begin) & (targeted.Year <= year_finish)]
    
    if count_type == 'unique_country':
        data_country = data_period[['Donor Name', 'CBPF Name']].drop_duplicates()
        donor_weight = pd.DataFrame(data_country.groupby(by=['Donor Name']).count()['CBPF Name'])
        donor_weight['transaction_sum'] = data_period.groupby(by='Donor Name').sum()['Total Contributions']
        donor_weight['transaction_number'] = data_period.groupby(by='Donor Name').count()['Total Contributions']
        donor_weight = donor_weight.reset_index()
        donor_weight['help_countries'] = ''

        poor_weight = pd.DataFrame(data_country.groupby(by=['CBPF Name']).count()['Donor Name'])
        poor_weight['transaction_sum'] = data_period.groupby(by='CBPF Name').sum()['Total Contributions']
        poor_weight['transaction_number'] = data_period.groupby(by='CBPF Name').count()['Total Contributions']
        poor_weight[['Total Allocations target', 'Targeted People target', 'Reached People target']] = targeted.groupby(by=['CBPF Name']).sum()[['Total Allocations', 'Targeted People', 'Reached People']]
        poor_weight[['Total Allocations affected', 'Targeted People affected', 'Reached People affected']] = targeted.groupby(by=['CBPF Name']).sum()[['Total Allocations', 'Targeted People', 'Reached People']]
        poor_weight = poor_weight.reset_index()
        poor_weight['help_countries'] = ''

        for donor in data_country['Donor Name'].unique():
            help_countries = data_country[data_country['Donor Name'] == donor]['CBPF Name'].values
            help_countries = ",".join(help_countries)
            donor_weight.loc[donor_weight['Donor Name'] == donor, 'help_countries'] = help_countries

        for poor in data_country['CBPF Name'].unique():
            help_countries = data_country[data_country['CBPF Name'] == poor]['Donor Name'].values
            help_countries = ",".join(help_countries)
            poor_weight.loc[poor_weight['CBPF Name'] == poor, 'help_countries'] = help_countries
        
        donor_weight['help_countries'] = donor_weight['help_countries'].apply(lambda x: x.split(','))
        poor_weight['help_countries'] = poor_weight['help_countries'].apply(lambda x: x.split(','))

        donor_weight = donor_weight.rename(columns={'CBPF Name': 'cbpf_number'})
        poor_weight = poor_weight.rename(columns={'Donor Name': 'donor_number'})

        return donor_weight, poor_weight



In [139]:
donor_weight, poor_weight = count_degree_paid(paid, affected, targeted, year_begin=2016, year_finish=2017)


In [140]:
poor_weight

,CBPF Name,donor_number,transaction_sum,transaction_number,Total Allocations target,Targeted People target,Reached People target,Total Allocations affected,Targeted People affected,Reached People affected,help_countries
0,Afghanistan,9,1.008450e+08,14,8.464083e+07,8251348,9208868,8.464083e+07,8251348,9208868,"[Australia, Denmark, Germany, Netherlands, Nor..."
1,CAR,10,5.671543e+07,19,4.480187e+07,5119711,4999610,4.480187e+07,5119711,4999610,"[Belgium, Canada, Denmark, Germany, Ireland, L..."
2,Colombia,4,2.706420e+06,7,3.290924e+06,28639,35994,3.290924e+06,28639,35994,"[Spain, Sweden, Switzerland, PRIVATE SECTOR]"
3,DRC,8,1.181444e+08,14,9.369797e+07,3726963,3449281,9.369797e+07,3726963,3449281,"[Belgium, Ireland, Luxembourg, Netherlands, Sw..."
4,Ethiopia,11,1.418558e+08,18,1.528589e+08,49865268,44495854,1.528589e+08,49865268,44495854,"[Australia, Denmark, Germany, Ireland, Korea, ..."
5,Iraq,13,1.875815e+08,21,1.648196e+08,10350693,12748891,1.648196e+08,10350693,12748891,"[Australia, Belgium, Canada, Cyprus, Denmark, ..."
6,Jordan,6,2.273346e+07,10,2.031299e+07,609766,658537,2.031299e+07,609766,658537,"[Belgium, Ireland, Netherlands, Sweden, United..."
7,Lebanon,9,2.530774e+07,13,3.319287e+07,706897,798524,3.319287e+07,706897,798524,"[Belgium, Germany, Ireland, Netherlands, Swede..."
8,Myanmar,8,1.592319e+07,12,1.535052e+07,627343,675639,1.535052e+07,627343,675639,"[Australia, Denmark, Sweden, Switzerland, Unit..."
9,Nigeria,17,4.369438e+07,17,2.395915e+07,2760274,3799839,2.395915e+07,2760274,3799839,"[Azerbaijan, Belgium, Canada, Denmark, Germany..."


In [125]:
affected.groupby(by=['CBPF Name']).sum()[['Total Allocations', 'Targeted People', 'Reached People']]

,Total Allocations,Targeted People,Reached People
CBPF Name,,,
Afghanistan,6.553381e+08,60164246,33552402
Burkina Faso (RhPF),1.994997e+07,1203905,0
CAR,2.223786e+08,19793886,18390785
Colombia,9.055308e+06,38725,48202
DRC,4.918896e+08,28054525,26376653
Ethiopia,5.329595e+08,103849396,92634240
Haiti,7.176283e+06,104600,0
Iraq,3.674129e+08,37546279,36067124
Jordan,6.166359e+07,2392671,2263613


In [117]:
targeted

,Unnamed: 0,Year,CBPF Name,Cluster,Total Allocations,Targeted People,Reached People
0,0,2013,Colombia,Emergency Shelter and NFI,1.079446e+05,0,0
1,1,2013,Colombia,Food Security,6.415703e+05,0,0
2,2,2013,Colombia,Water Sanitation Hygiene,1.500000e+05,0,0
3,3,2013,DRC,Coordination and Support Services,5.082700e+05,20259,0
4,4,2013,DRC,Education,6.650000e+05,32091,0
...,...,...,...,...,...,...,...
1250,104,2022,Venezuela,Nutrition,1.880679e+04,171,0
1251,105,2022,Venezuela,Protection,1.241345e+06,21884,0
1252,106,2022,Venezuela,Water Sanitation Hygiene,2.711754e+05,3037,0
1253,107,2022,Yemen,Multi-Sector,3.011348e+06,275800,0
